In [1]:
import sys
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
sys.path.append("..")

# news scraping
from src.zinfo.news_scraper import NewsScraper

# clustering
from src.zinfo.article_clustering import get_vectorized_titles
from src.zinfo.article_clustering import get_num_clusters_per_val
from src.zinfo.article_clustering import cluster_articles

# article selecting
from src.zinfo.article_selector import get_best_article_all_clusters

In [50]:
def switch_columns(df, col1, col2):
    cols = list(df.columns)
    a, b = cols.index(col1), cols.index(col2)
    cols[b], cols[a] = cols[a], cols[b]
    dframe = df[cols]
    return dframe

In [3]:
start_date = "2021-08-13"
selected_news = pd.read_csv("../data/selected_articles.csv")
todays_articles = selected_news.loc[selected_news.date >= start_date]

In [63]:
all_news = pd.read_csv("../data/all_news.csv")
selected_news = pd.read_csv("../data/selected_articles.csv")

In [3]:
clustered_news = cluster_articles(all_news)

clustering articles...
eps_val: 0.22
min_samples: 3 
target cluster num: 14


In [4]:
summarized_news = get_best_article_all_clusters(clustered_news, all_news)

selecting best articles out of each cluster
